In [34]:
import random
import numpy as np
import matplotlib.pyplot as plt
from lxml import etree
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from itertools import permutations, combinations
from random import shuffle
import statistics
import seaborn as sns

In [ ]:
# Function to parse the XML file and extract city data
def parse_xml(xml_file):
    city_data = []
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for vertex_elem in root.findall(".//vertex"):
        edge_costs = [float(edge.get("cost")) for edge in vertex_elem]
        city_data.append(edge_costs)

    return city_data

In [ ]:
 cities = parse_xml("burma14.xml")

In [ ]:
def generate_population(population_size):
        population = []
        for _ in range(population_size):
            individual = random.sample(range(num_cities), num_cities)
            population.append(individual)
        return population

In [ ]:
 def select_parents(population, tournament_size):
        parents = []
        for _ in range(2):  # Select two parents
            tournament = random.sample(population, tournament_size)
            best_individual = min(tournament, key=lambda x: tour_distance(x, distances))
            parents.append(best_individual)
        return parents

In [ ]:
def crossover_fixed_point(parent1, parent2):
    crossover_point = random.randint(1, len(parent1)-1)
    child1 = parent1[:crossover_point] + [c for c in parent2 if c not in parent1[:crossover_point]]
    child2 = parent2[:crossover_point] + [c for c in parent1 if c not in parent2[:crossover_point]]
    return child1, child2


In [ ]:
def mutate_single_swap(individual):
    # Convert the individual tuple to a list
    
    individual_list = list(individual)
  
    # Select two random positions in the chromosome
    position1 = random.randint(0, len(individual) - 1)
    position2 = random.randint(0, len(individual) - 1)

    # Swap the genes at the selected positions
    individual_list[position1], individual_list[position2] = individual_list[position2], individual_list[position1]

    return individual_list

In [ ]:
def evaluate_tsp_fitness(individual):
    total_distance = tour_distance(individual, distances)
    cost = total_distance
    fitness = cost  # Invert the cost to make smaller costs more fit
    return fitness

In [ ]:
def mutate_inversion(individual):
    
    # Select two random positions in the chromosome
    position1 = random.randint(0, len(individual) - 1)
    position2 = random.randint(0, len(individual) - 1)

    # Ensure position1 is smaller than position2
    if position1 > position2:
        position1, position2 = position2, position1

    # Reverse the subsequence between position1 and position2
    individual[position1:position2+1] = reversed(individual[position1:position2+1])

    return individual

In [ ]:
def crossover_ordered (parent1, parent2):
    """Implements ordered crossover"""

    size = len(parent1.vertices)

    # Choose random start/end position for crossover
    child1, child2 = [-1] * size, [-1] * size
    start, end = sorted([random.randrange(size) for _ in range(2)])

    # Replicate mum's sequence for alice, dad's sequence for bob
    child1_inherited = []
    child2_inherited = []
    for i in range(start, end + 1):
        child1[i] = parent1.vertices[i]
        child2[i] = parent2.vertices[i]
        child1_inherited.append(parent1.vertices[i])
        child2_inherited.append(parent2.vertices[i])

    print(child1, child2)
    #Fill the remaining position with the other parents' entries
    current_parent2_position, current_parent1_position = 0, 0

    fixed_pos = list(range(start, end + 1))       
    i = 0
    while i < size:
        if i in fixed_pos:
            i += 1
            continue

        test_child1 = child1[i]
        if test_child1==-1: #to be filled
            parent2_trait = parent2.vertices[current_parent2_position]
            while parent2_trait in child1_inherited:
                current_parent2_position += 1
                parent2_trait = parent2.vertices[current_parent2_position]
            child1[i] = parent2_trait
            child1_inherited.append(parent2_trait)

        #repeat block for bob and mom
        i +=1

    return child1, child2



In [ ]:
# Define EA parameters
population_size = 100
num_generations = 1
tournament_size = 4
mutation_rate = 0.1

In [ ]:
# Experiment 1: Varying Crossover Operator

# Defined population and other parameters
crossover_operator = crossover_fixed_point
mutation_operator = mutate_single_swap

population = generate_population(population_size)

best_distance = float('inf')
best_tour = None
distances_log1=[]
experiment1_results = []

for generation in range(num_generations):
        new_population = []
        for _ in range(population_size):
            # Selection
            parent1, parent2 = select_parents(population, tournament_size)
            # Crossover
            child = crossover_operator(parent1, parent2)
            # Mutation
            if random.random() < mutation_rate:
                child = mutation_operator(child)
            # Evaluate fitness
            fitness = evaluate_tsp_fitness(child)
            new_population.append((child, fitness))
        population = new_population
        
for individual in population:
        distance = tour_distance(individual, distances)
        if distance < best_distance:
            best_distance = distance
            best_tour = individual
            distances_log1.append(best_distance)
            experiment1_results.append(best_distance)

In [ ]:
# Experiment 2: Varying Mutation Operator
# Defined population and other parameters
crossover_operator = crossover_ordered  
mutation_operator = mutate_inversion 


population = generate_population(population_size)

for generation in range(num_generations):
        new_population = []
        for _ in range(population_size):
            # Selection
            parent1, parent2 = select_parents(population, tournament_size)
            # Crossover
            child = crossover_operator(parent1, parent2)
            # Mutation
            if random.random() < mutation_rate:
                child = mutation_operator(child)
            # Evaluate fitness
            fitness = evaluate_tsp_fitness(child)
            new_population.append((child, fitness))
        population = new_population

In [ ]:
# Experiment 1: Varying Crossover Operator

# Defined population and other parameters
crossover_operator = crossover_fixed_point
mutation_operator = mutate_single_swap


best_distance = float('inf')
best_tour = None
distances_log1=[]
experiment1_results = []


for _ in range(num_generations):
    new_population = []
    for _ in range(population_size):
        parent1, parent2 = random.sample(population, 2)
        child1, child2 = crossover_fixed_point(parent1, parent2)
        if random.random() < mutation_rate:
            child1 = mutate_single_swap(child1)
        if random.random() < mutation_rate:
            child2 = mutate_single_swap(child2)
        new_population.extend([child1, child2])

    population = new_population

    # Calculate and store the best distance in each generation
    best_distance = float('inf')
    best_tour = None
    distance = tour_distance(individual, distances)

            
    for individual in population:
        distance = tour_distance(individual, distances)
        if distance < best_distance:
            best_distance = distance
            best_tour = individual

    distances_log1.append(best_distance)
    experiment1_results.append(best_distance)


In [ ]:
print("Best distance:", experiment1_results)

In [ ]:
# Experiment 2: Varying Mutation Operator
# Defined population and other parameters

# Initialize a list to store results

best_distance = float('inf')
best_tour = None
distances_log2=[]
experiment2_results = []
for _ in range(num_generations):
    # Example code for EA generation    
    new_population = []
    for _ in range(population_size):
        parent1, parent2 = random.sample(population, 2)
        child1, child2 = crossover_ordered(parent1, parent2)
        if random.random() < mutation_rate:
            child1 = mutate_inversion(child1)
        if random.random() < mutation_rate:
            child2 = mutate_inversion(child2)
        new_population.extend([child1, child2])

    population = new_population

    # Calculate and store the best distance in each generation
    best_distance = float('inf')
    best_tour = None
    distance = tour_distance(individual, distances)

            
    for individual in population:
        distance = tour_distance(individual, distances)
        if distance < best_distance:
            best_distance = distance
            best_tour = individual

    distances_log2.append(best_distance)
    experiment2_results.append(best_distance)    



In [ ]:
print("Best distance:", experiment2_results)